In [1]:
import copy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
import time
import sys, io, os

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [3]:
import json
from tqdm import tqdm

In [4]:
from torchvision import datasets, transforms

In [5]:
device = torch.device('cuda:0')

## Data Loader

In [6]:
# cifar_train = transforms.Compose([
#     transforms.RandomCrop(size=32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
#         std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
#     ),
# ])

# cifar_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
#         std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
#     ),
# ])

# train_dataset = datasets.CIFAR10(root="../../_Datasets/cifar10/", train=True, download=True, transform=cifar_train)
# test_dataset = datasets.CIFAR10(root="../../_Datasets/cifar10/", train=False, download=True, transform=cifar_test)

In [38]:
cifar_train = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4865, 0.4409],
        std=[0.2009, 0.1984, 0.2023],
    ),
])

cifar_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4865, 0.4409],
        std=[0.2009, 0.1984, 0.2023],
    ),
])

train_dataset = datasets.CIFAR100(root="../../_Datasets/cifar100/", train=True, download=False, transform=cifar_train)
test_dataset = datasets.CIFAR100(root="../../_Datasets/cifar100/", train=False, download=False, transform=cifar_test)

In [39]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=128, shuffle=False, num_workers=4)

In [40]:
len(train_dataset.classes)

100

## Model

In [41]:
import cifar_resnet_custom as resnet

In [46]:
inplanes=9 ## 15, 17
# model = resnet.cifar_resnet20(inplanes=inplanes)
model = resnet.cifar2_resnet20(inplanes=inplanes, num_classes=len(train_dataset.classes))

In [47]:
n_params = np.sum([p.numel() for p in model.parameters()])
print(f"Number of parameters is: {n_params}") ## 1148152

Number of parameters is: 229537


In [48]:
model(torch.randn(1, 3, 32, 32))

tensor([[-0.7695, -0.3091, -1.0481,  0.2146,  0.3699, -0.5418,  0.9250,  0.3687,
         -0.0968,  0.1168,  0.2080,  0.0733,  0.1434, -0.7162,  0.1073, -0.6380,
         -0.1837, -0.4494,  0.3833,  0.6699,  0.2160,  0.3846, -0.3169, -0.1343,
          0.4922,  0.0109, -0.7599, -0.1888,  0.2535,  0.2659, -0.0281,  0.0250,
         -0.8213,  0.3539,  0.4358,  0.3366,  0.0231, -0.0819,  0.8935, -0.4142,
         -0.2439,  0.4960, -0.2321, -0.1786,  0.6740, -0.1887, -0.4778, -0.2186,
          0.8339,  0.4513, -0.0202, -0.1833,  0.0074,  0.1142, -0.1035, -0.1023,
         -0.6672,  0.4931,  0.3743,  0.4675, -0.1425, -0.7868, -1.0239, -0.1973,
         -0.0266, -0.0953, -0.1525, -0.0374, -0.3004, -0.4754, -0.5070,  0.6558,
          0.7559,  0.1253, -0.1238,  0.5136,  1.0662, -0.0970,  0.8671, -0.2496,
          0.6353, -0.1433, -0.5208,  0.3894, -0.0195,  0.5475, -0.5269, -0.1806,
         -1.0206,  0.2613, -0.7348, -0.2904,  0.0648,  0.3859, -0.3716,  0.2340,
         -0.0150,  0.0068,  

In [49]:
model.to(device)

CifarResNet2(
  (conv1): Sequential(
    (0): Conv2d(3, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): Dropout2d(p=0.1, inplace=False)
  )
  (bn1): BatchNorm2d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2d(9, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): Dropout2d(p=0.1, inplace=False)
      )
      (bn1): BatchNorm2d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Sequential(
        (0): Conv2d(9, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): Dropout2d(p=0.1, inplace=False)
      )
      (bn2): BatchNorm2d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv2d(9, 9, kernel_size=(3, 3), stride=(1, 1), p

In [50]:
asfsdfadf

NameError: name 'asfsdfadf' is not defined

## Optimizer and Loss

In [51]:
EPOCHS = 200

In [52]:
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1,
#                       momentum=0.9, weight_decay=5e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [53]:
# model_name = f"resnet20_inplanes_{inplanes}_params_{n_params}.pth" ## default bs 32
# model_name = f"resnet20_b4_inplanes_{inplanes}_params_{n_params}.pth" ## b4 uses cifar2_resnet
model_name = f"resnet20_b4_c100_inplanes_{inplanes}_params_{n_params}_bs128.pth"

In [54]:
train_accs = []
test_accs = []
train_losses, test_losses = [], []

## Training Code

In [55]:
## Following is copied from 
### https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

# Training
def train(epoch):
#     print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print(f"[Train] {epoch} Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    train_accs.append(100.*correct/total)
    train_losses.append(train_loss/(batch_idx+1))
    return

In [56]:
best_acc = -1
def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(test_loader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
    print(f"[Test] {epoch} Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    test_losses.append(test_loss/(batch_idx+1))
    
    # Save checkpoint.
    acc = 100.*correct/total
    test_accs.append(acc)
    if acc > best_acc:
        print('Saving..')
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f'./models/{model_name}')
        best_acc = acc

In [57]:
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
resume = False

if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load(f'./models/{model_name}')
    model.load_state_dict(checkpoint['model'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

In [58]:
def save_accs():
    with open(f"./models/{model_name}_accs_loss.json", 'w') as f:
        d = {
            "train_accs":train_accs,
            "test_accs":test_accs,
            "train_loss":train_losses,
            "test_loss":test_losses,
        }
        json.dump(d, f, indent=0)

In [59]:
### Train the whole damn thing

for epoch in range(start_epoch, start_epoch+EPOCHS): ## for 200 epochs
    train(epoch)
    test(epoch)
    scheduler.step()
    save_accs()

100%|███████████████████████████████████████████████████| 391/391 [00:07<00:00, 52.93it/s]


[Train] 0 Loss: 4.309 | Acc: 3.808 1904/50000


100%|████████████████████████████████████████████████████| 79/79 [00:00<00:00, 143.62it/s]


[Test] 0 Loss: 3.983 | Acc: 7.730 773/10000
Saving..


100%|███████████████████████████████████████████████████| 391/391 [00:07<00:00, 53.23it/s]


...

[Test] 198 Loss: 1.585 | Acc: 56.960 5696/10000


100%|███████████████████████████████████████████████████| 391/391 [00:07<00:00, 52.83it/s]


[Train] 199 Loss: 1.718 | Acc: 52.476 26238/50000


100%|████████████████████████████████████████████████████| 79/79 [00:00<00:00, 140.98it/s]

[Test] 199 Loss: 1.581 | Acc: 56.960 5696/10000


In [60]:
adsrsrs

NameError: name 'adsrsrs' is not defined

In [61]:
best_acc, model_name

(57.18, 'resnet20_b4_c100_inplanes_9_params_229537_bs128.pth')

### Load Best Model State

In [16]:
print('==> Resuming from checkpoint..')
assert os.path.isdir('models'), 'Error: no checkpoint directory found!'
checkpoint = torch.load(f'./models/{model_name}')
model.load_state_dict(checkpoint['model'])
best_acc = checkpoint['acc']
start_epoch = checkpoint['epoch']

==> Resuming from checkpoint..


In [17]:
### Linear -> (92.96, 184)
### Distance -> (92.64, 199)
best_acc, start_epoch

(87.87, 159)

## Experiment Logs

In [ ]:
### Resnet20 , wo Dropout, inplanes=11, --> (89.21, 'resnet_20_inplanes_11_params_129359.pth')
### Resnet20 , w Dropout, inplanes=11, --> (89.64, 'resnet20_inplanes_11_params_129359.pth')

### Resnet20, w Dropout, inplanes=21, 4 blocks --> (93.2, 'resnet20_b4_inplanes_21_params_1206691.pth')
### Resnet20, w Dropout, inplanes=21, 3 blocks --> (94.74, 'resnet20_inplanes_33_params_1153261.pth')

### Resnet20, w Dropout, inplanes=9, 4 blocks --> (87.87, 'resnet20_b4_inplanes_9_params_222967.pth')
### Resnet20, w Dropout, inplanes=13, 4 blocks --> (90.51, 'resnet20_b4_inplanes_13_params_463707.pth')
### Resnet20, w Dropout, inplanes=17, 4 blocks --> (92.25, 'resnet20_b4_inplanes_17_params_791615.pth')